# 🌞 Example Weather App
> Using `prodb`


* Starting with a `.csv` of locations, we wish to call the [MetaWeather](https://www.metaweather.com/api/) API to pull in the weather forecast.  
* The app needs the ability to input new locations by the user

 🌍 Core functions will be converted into the streamlit app `weather_app.py`

In [ ]:
#hide
from nbdev.showdoc import *
import pandas as pd
%load_ext autoreload
%autoreload 2

## 1. Requests to `metaweather` open weather API

In [ ]:
import requests
import json
import arrow

In [ ]:
def get_json_from_query(location):
    """Search for a city and return metadata from API"""
    url = f"https://www.metaweather.com/api/location/search/?query={location}"
    r = requests.get(url).json()
    return r[0]

l = get_json_from_query('London')
l

{'title': 'London',
 'location_type': 'City',
 'woeid': 44418,
 'latt_long': '51.506321,-0.12714'}

Check `woeid`(Where On Earth ID) is correct for London

In [ ]:
assert l['woeid'] == 44418

Get weather data for the London `woeid`

In [ ]:
utc = arrow.utcnow().format('YYYY/MM/DD')
utc

'2021/10/03'

In [ ]:
def get_current_weather(location):
    """
    inputs: location str "London"
            to find woeid i.e. 44418
    """
    res = get_json_from_query(location)
    woeid = res['woeid']
    url = f"https://www.metaweather.com/api/location/{woeid}/"
    res = requests.get(url).json()
    return res

Today's forecast:

In [ ]:
res = get_current_weather('London')
res['consolidated_weather'][0]

{'id': 5495230276042752,
 'weather_state_name': 'Light Rain',
 'weather_state_abbr': 'lr',
 'wind_direction_compass': 'SW',
 'created': '2021-10-03T12:59:03.049755Z',
 'applicable_date': '2021-10-03',
 'min_temp': 10.645,
 'max_temp': 16.015,
 'the_temp': 14.4,
 'wind_speed': 8.614009459582704,
 'wind_direction': 227.33346500159422,
 'air_pressure': 997.0,
 'humidity': 75,
 'visibility': 11.97755110156685,
 'predictability': 75}

In [ ]:
res

{'consolidated_weather': [{'id': 5495230276042752,
   'weather_state_name': 'Light Rain',
   'weather_state_abbr': 'lr',
   'wind_direction_compass': 'SW',
   'created': '2021-10-03T12:59:03.049755Z',
   'applicable_date': '2021-10-03',
   'min_temp': 10.645,
   'max_temp': 16.015,
   'the_temp': 14.4,
   'wind_speed': 8.614009459582704,
   'wind_direction': 227.33346500159422,
   'air_pressure': 997.0,
   'humidity': 75,
   'visibility': 11.97755110156685,
   'predictability': 75},
  {'id': 5728701745463296,
   'weather_state_name': 'Showers',
   'weather_state_abbr': 's',
   'wind_direction_compass': 'SSW',
   'created': '2021-10-03T12:59:03.153137Z',
   'applicable_date': '2021-10-04',
   'min_temp': 11.245000000000001,
   'max_temp': 16.665,
   'the_temp': 16.035,
   'wind_speed': 8.81605335446857,
   'wind_direction': 213.3203208686136,
   'air_pressure': 1009.5,
   'humidity': 66,
   'visibility': 11.50655173785095,
   'predictability': 73},
  {'id': 6058180229464064,
   'weather

There are a number of metrological properties available to us from the API. 

The ones we are most interested in are:
* `the_temp` 🌡️ current temperature
* `weather_state_name` ⛅ current sky condition
* `max_temp` 🥵 daily max temp
* `min_temp` 🥶 daily min temp


In [ ]:
res = get_current_weather('London')
df =  pd.DataFrame.from_records(res['consolidated_weather'][:1])
df['readable_time'] = df.created.apply(lambda x: arrow.get(x).humanize())
df

,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability,readable_time
0,5495230276042752,Light Rain,lr,SW,2021-10-03T12:59:03.049755Z,2021-10-03,10.645,16.015,14.4,8.614009,227.333465,997.0,75,11.977551,75,2 hours ago


In [ ]:
def df_from_loc(location, 
                days_ahead=1,
                keep_cols='location the_temp readable_time created applicable_date local_time latt_long weather_state_name min_temp max_temp'.split(' ')):
    res = get_current_weather(location)
    df =  pd.DataFrame.from_records(res['consolidated_weather'][:days_ahead])
    df['location'] = location
    df['local_time'] = res['time']
    df['latt_long'] = res['latt_long']
    df['readable_time'] = df.created.apply(lambda x: arrow.get(x).humanize())
    return df[keep_cols]

df = df_from_loc('London', days_ahead=2)
df

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,London,14.400,2 hours ago,2021-10-03T12:59:03.049755Z,2021-10-03,2021-10-03T16:19:50.788861+01:00,"51.506321,-0.12714",Light Rain,10.645,16.015
1,London,16.035,2 hours ago,2021-10-03T12:59:03.153137Z,2021-10-04,2021-10-03T16:19:50.788861+01:00,"51.506321,-0.12714",Showers,11.245,16.665


Example output showing location and current weather situation

In [ ]:
df = df_from_loc('London')
df

,location,the_temp,readable_time,created,applicable_date,weather_state_name,min_temp,max_temp
0,London,14.4,2 hours ago,2021-10-03T12:59:03.049755Z,2021-10-03,Light Rain,10.645,16.015


In [ ]:
df = df_from_loc('Auckland')
df

,location,the_temp,readable_time,created,applicable_date,weather_state_name,min_temp,max_temp
0,Auckland,16.995,an hour ago,2021-10-03T13:37:05.475920Z,2021-10-04,Light Rain,13.15,18.775


In [ ]:
loc = get_json_from_query('Auckland')
res = get_current_weather(l['woeid'])
res[0]